In [2]:
import sys
import os
from pathlib import Path
import json

# Obtener la ruta absoluta a la carpeta raíz del proyecto
ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))


# Añadir esa ruta a sys.path
if ROOT not in sys.path:
    sys.path.append(ROOT)

print(ROOT)

# print(os.path.isfile(PROMPTS_DB_PATH))

c:\Formacion\Customer_bot


In [3]:
from bot.config import PROMPTS_DB_PATH, OPENAI_MODEL_NAME, OPENAI_API_KEY
from bot.models import BotState
from bot.nlp_agent import nlp_node
from bot.graph import initial_state

PROMPTS_DB_PATH = os.path.join(ROOT, PROMPTS_DB_PATH)

# Añadir esa ruta a sys.path
if PROMPTS_DB_PATH not in sys.path:
    sys.path.append(PROMPTS_DB_PATH)

print(PROMPTS_DB_PATH)

c:\Formacion\Customer_bot\db/prompts.db


In [4]:
path = Path(PROMPTS_DB_PATH)
if path.exists():
    with path.open("r", encoding="utf-8") as f:
        prompts = json.load(f)
        print("Claves en prompts:", list(prompts.keys()))
else:
    print("Fichero no encontrado.")

system_prompt = prompts["nlp_agent.system"]
prompt_answer = prompts["answer_agent.system"]

Claves en prompts: ['nlp_agent.system', 'answer_agent.system']


In [5]:
from bot.models import BotState, IntentResult, KnowledgeHit, Entity, AnswerResult, NLPResult
from bot.answer import answer_node, build_context_text

In [6]:
intent = IntentResult(
    tipo_mensaje="pregunta",
    intencion="canarias",
    confianza=0.92,
    sentimiento="neutral"
)

ent1 = Entity(tipo="producto", valor="auriculares")
ent2 = Entity(tipo="numero_pedido", valor="XYZ123")
ent3 = Entity(tipo="ciudad", valor="Madrid")

nlp_result = NLPResult(
    intent=intent,
    entidades=[ent1, ent2, ent3]
)

hit = KnowledgeHit(
    categoria="informacion",
    pregunta_canonica="¿Dame información sobre este producto?",
    respuesta_base="Son unos auriculares maravillosos.",
    score=1.0
)

answer = AnswerResult(
    respuesta="Tu pedido llegará mañana.",
    necesita_revision_humano=False,
    razon=None,
    metadata={"fuente": "FAQ base"}
)

state =  BotState(
    user_message="¿Dónde está mi pedido?",
    nlp = nlp_result,
    knowledge_hits=[hit],
    answer=answer,
    debug={}
)

In [ ]:
user_message = state.user_message
nlp = state.nlp
hits = state.knowledge_hits

print(user_message)
print(nlp.model_dump_json(indent=4, ensure_ascii=False))
for hit in hits:
    print(hit)
    print(hit.model_dump_json(indent=4, ensure_ascii=False))

¿Dónde está mi pedido?
{
    "intent": {
        "tipo_mensaje": "pregunta",
        "intencion": "canarias",
        "confianza": 0.92,
        "sentimiento": "neutral"
    },
    "entidades": [
        {
            "tipo": "producto",
            "valor": "auriculares"
        },
        {
            "tipo": "numero_pedido",
            "valor": "XYZ123"
        },
        {
            "tipo": "ciudad",
            "valor": "Madrid"
        }
    ]
}
categoria='informacion' pregunta_canonica='¿Dame información sobre este producto?' respuesta_base='Son unos auriculares maravillosos.' score=1.0
{
    "categoria": "informacion",
    "pregunta_canonica": "¿Dame información sobre este producto?",
    "respuesta_base": "Son unos auriculares maravillosos.",
    "score": 1.0
}


In [8]:
ctx_lines = []
for hit in hits:
        linea = f"[{hit.categoria}] {hit.pregunta_canonica}: {hit.respuesta_base}"
        ctx_lines.append(linea)

In [11]:
contexto = build_context_text(hits)

  # Para inspección / depuración, a veces es útil tener el NLP en JSON plano
nlp_json = nlp.model_dump_json(indent=4, ensure_ascii=False)

In [12]:
user_prompt = f"""
MENSAJE DEL CLIENTE:
{user_message}

INTENCIÓN DETECTADA (JSON):
{nlp_json}

BASE DE CONOCIMIENTO:
{contexto}

INSTRUCCIONES:
- Responde al cliente usando SOLO la información de BASE DE CONOCIMIENTO.
- Si falta información para responder con seguridad, dilo claramente y sugiere derivar el caso a un agente humano.
- No inventes políticas, plazos ni condiciones que no estén en el contexto.
- Mantén un tono profesional, cercano y claro.
"""